### 🎯 Hand Pose Estimation - 모델 테스트 (Testing)

이 노트북에서는 학습된 모델을 테스트하고 성능을 평가합니다.

#### 📚 학습 목표
- 학습된 모델을 불러와서 테스트 데이터에 적용하기
- 모델의 성능 지표 분석하기
- 다양한 임계값(threshold)으로 실험하며 결과 비교하기
- 예측 결과를 시각화하고 분석하기

---
#### 1️⃣ 필요한 라이브러리 불러오기

In [1]:
# 필요한 라이브러리 불러오기
import os
import warnings
warnings.filterwarnings('ignore')
from ultralytics import YOLO
from ultralytics import settings
import cv2
import matplotlib.pyplot as plt
from IPython.display import Image, display
import numpy as np

print("✅ 라이브러리 로딩 완료!")

WARNING ⚠️ Python>=3.10 is required, but Python==3.8.20 is currently installed 
✅ 라이브러리 로딩 완료!


---
#### 2️⃣ 경로 설정

In [2]:
# 현재 작업 디렉토리 확인
current_dir = os.getcwd()
print(f"현재 작업 경로: {current_dir}")

# 테스트 결과 저장 경로 설정
new_runs_dir = os.path.abspath(os.path.join(current_dir, '../runs'))
new_data_dir = os.path.abspath(os.path.join(current_dir, '../data_10%'))

# YOLO 설정 업데이트
settings.update({"runs_dir": new_runs_dir})
settings.update({"datasets_dir": new_data_dir})
settings.update({"wandb": False})

print(f"테스트 결과 저장 경로: {new_runs_dir}")
print(f"데이터셋 경로: {new_data_dir}")
print("\n✅ 경로 설정 완료!")

현재 작업 경로: /home/nute11a/workspace/2026_AI_Advanced_Study/3차시/03_Hand_Detection/code
테스트 결과 저장 경로: /home/nute11a/workspace/2026_AI_Advanced_Study/3차시/03_Hand_Detection/runs
데이터셋 경로: /home/nute11a/workspace/2026_AI_Advanced_Study/3차시/03_Hand_Detection/data_10%

✅ 경로 설정 완료!


---

#### 3️⃣ 학습된 모델 불러오기 🔍

- train.ipynb에서 학습한 모델의 경로를 찾아서 아래 `MODEL_PATH` 변수에 입력해주세요.

**힌트:**
- 보통 `../runs/pose/train/weights/best.pt` 또는 `../runs/pose/train/weights/last.pt`에 저장됩니다.
- `best.pt`: 검증 성능이 가장 좋았던 모델 (추천!)
- `last.pt`: 마지막 epoch의 모델


In [3]:
# ==========================================
# 🔧 여기를 수정하세요!
# ==========================================

# TODO: 학습된 모델의 경로를 입력하세요
# 예시: MODEL_PATH = "../runs/pose/train/weights/best.pt"
MODEL_PATH = "../runs/pose/train8/weights/best.pt" #""  # ← 여기를 수정!

# ==========================================
if os.path.exists(MODEL_PATH):
    model = YOLO(MODEL_PATH)
    print("✅ 모델 로딩 완료!")
else:
    print(f"❌ 모델 파일을 찾을 수 없습니다: {MODEL_PATH}")

✅ 모델 로딩 완료!


---
#### 4️⃣ 모델 검증 실행 🚀


In [4]:
DATA_CONFIG = "../data_10%/config.yaml"

metrics = model.val(
    data=DATA_CONFIG,
    split = 'test',
    plots=True,  # 검증 그래프 생성
)
print("\n✅ 검증 완료!")

Ultralytics 8.3.11 🚀 Python-3.8.20 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12007MiB)


YOLO11n-pose summary (fused): 257 layers, 2,956,000 parameters, 0 gradients, 7.8 GFLOPs


val: Scanning /home/nute11a/workspace/2026_AI_Advanced_Study/3차시/03_Hand_Detection/data_10%/labels/test.cache... 522 images, 0 backgrounds, 0 corrupt: 100%|██████████| 522/522 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:02<00:00, 16.04it/s]


                   all        522        522      0.983      0.975      0.992      0.862      0.812      0.709      0.748      0.502
Speed: 0.3ms preprocess, 1.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /home/nute11a/workspace/2026_AI_Advanced_Study/3차시/03_Hand_Detection/runs/pose/val11

✅ 검증 완료!


---
#### 5️⃣ 성능 지표 분석 📊

테스트 데이터셋에 대해 모델의 성능을 평가합니다.

**평가 지표 설명:**
- **Precision (정밀도)**: 모델이 "손"이라고 예측한 것 중 실제로 손인 비율
- **Recall (재현율)**: 실제 손 중에서 모델이 찾아낸 비율
- **mAP50**: IoU 50% 기준으로 계산한 평균 정밀도
- **mAP50-95**: IoU 50%~95% 범위에서 계산한 평균 정밀도 (더 엄격한 기준)

In [5]:
print("="*60)
print("📊 모델 성능 요약")
print("="*60)
# Box Detection 성능
print("\n🔲 손 위치 감지 (Bounding Box) 성능:")
print(f"  - Precision: {metrics.box.p[0]:.4f}")
print(f"  - Recall: {metrics.box.r[0]:.4f}")
print(f"  - mAP50: {metrics.box.map50:.4f}")
print(f"  - mAP50-95: {metrics.box.map:.4f}")
# Pose Estimation 성능
print("\n🖐️ 손 자세 추정 (Keypoint) 성능:")
print(f"  - Precision: {metrics.pose.p[0]:.4f}")
print(f"  - Recall: {metrics.pose.r[0]:.4f}")
print(f"  - mAP50: {metrics.pose.map50:.4f}")
print(f"  - mAP50-95: {metrics.pose.map:.4f}")
print("\n" + "="*60)

📊 모델 성능 요약

🔲 손 위치 감지 (Bounding Box) 성능:
  - Precision: 0.9833
  - Recall: 0.9751
  - mAP50: 0.9925
  - mAP50-95: 0.8620

🖐️ 손 자세 추정 (Keypoint) 성능:
  - Precision: 0.8116
  - Recall: 0.7088
  - mAP50: 0.7475
  - mAP50-95: 0.5016



#### 6️⃣ 전체 데이터로 학습한 모델과 비교하기 👨‍🏫

우리는 시간 관계상 전체 데이터의 `10%`만 사용하여 모델을 학습했습니다.
그렇다면, `전체 데이터(100%)`를 모두 학습한 모델은 얼마나 더 똑똑할까요?

미리 학습된 `전체 데이터셋에서 학습된 Model`을 불러와서 성능을 비교해 봅시다.

In [6]:
# 1. 전체 데이터로 학습된 모델 경로 설정
# (강사님이 미리 학습시켜둔 모델 파일의 경로를 지정해주세요)
FULL_MODEL_PATH = "../weights/pose_full_train_model.pt"  # 예시 경로 (실제 파일 위치로 수정 필요!)

full_model = YOLO(FULL_MODEL_PATH)

print("\n▶️ Teacher Model 성능 평가 중...")
full_model_metrics = full_model.val(data=DATA_CONFIG, split = 'test', plots=False, verbose=False)

print("="*60)
print("📊 모델 성능 요약")
print("="*60)
# Box Detection 성능
print("\n🔲 손 위치 감지 (Bounding Box) 성능:")
print(f"  - Precision: {full_model_metrics.box.p[0]:.4f}")
print(f"  - Recall: {full_model_metrics.box.r[0]:.4f}")
print(f"  - mAP50: {full_model_metrics.box.map50:.4f}")
print(f"  - mAP50-95: {full_model_metrics.box.map:.4f}")
# Pose Estimation 성능
print("\n🖐️ 손 자세 추정 (Keypoint) 성능:")
print(f"  - Precision: {full_model_metrics.pose.p[0]:.4f}")
print(f"  - Recall: {full_model_metrics.pose.r[0]:.4f}")
print(f"  - mAP50: {full_model_metrics.pose.map50:.4f}")
print(f"  - mAP50-95: {full_model_metrics.pose.map:.4f}")
print("\n" + "="*60)


▶️ Teacher Model 성능 평가 중...
Ultralytics 8.3.11 🚀 Python-3.8.20 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12007MiB)


YOLO11n-pose summary (fused): 257 layers, 2,956,000 parameters, 0 gradients, 7.8 GFLOPs


val: Scanning /home/nute11a/workspace/2026_AI_Advanced_Study/3차시/03_Hand_Detection/data_10%/labels/test.cache... 522 images, 0 backgrounds, 0 corrupt: 100%|██████████| 522/522 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:01<00:00, 25.58it/s]


                   all        522        522      0.987       0.99      0.995      0.939      0.932      0.915      0.936      0.818
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 0.2ms postprocess per image
📊 모델 성능 요약

🔲 손 위치 감지 (Bounding Box) 성능:
  - Precision: 0.9866
  - Recall: 0.9898
  - mAP50: 0.9946
  - mAP50-95: 0.9388

🖐️ 손 자세 추정 (Keypoint) 성능:
  - Precision: 0.9317
  - Recall: 0.9147
  - mAP50: 0.9361
  - mAP50-95: 0.8179



---
##### 🤔 **생각하기: 왜 데이터가 많을수록 성능이 좋아질까?**

위 그래프를 보면 10% 데이터로 학습한 모델보다 **전체 데이터로 학습한 모델의 성능이 더 높은 것**을 확인할 수 있습니다. (만약 차이가 크지 않다면, 10% 데이터만으로도 충분히 학습이 잘 되었다는 뜻이기도 합니다!)

##### **왜 그럴까요?**

1.  **다양한 경험 (Generalization):**
    * AI는 데이터를 통해 세상을 배웁니다.
    * 10%의 데이터에는 특정 각도나 조명의 손 사진만 있을 수 있습니다.
    * 전체 데이터에는 더 다양한 손 모양, 배경, 밝기 등의 케이스가 포함되어 있어, AI가 **"어떤 상황에서도(Generalization)"** 손을 더 잘 찾게 됩니다.

2.  **과적합 방지 (Overfitting):**
    * 공부할 양이 너무 적으면, AI는 문제집의 답을 단순히 외워버릴 수 있습니다(과적합).
    * 데이터가 많으면 단순 암기가 불가능해져서, AI는 손의 **"진짜 특징(패턴)"**을 찾아내려고 노력하게 됩니다.

> 💡 결론: 좋은 AI를 만들기 위해서는 `좋은 모델(알고리즘)`도 중요하지만, `양질의 데이터(Data)`를 많이 확보하는 것이 무엇보다 중요합니다!